# Import Package

In [1]:
import string
import math
import re
import numpy as np
import collections
import matplotlib.pyplot as plt
from stemming.porter2 import stem
from collections import Counter
import numpy as np
# from gensim.test.utils import common_texts
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

# Word-Level Comparisons

In [2]:
with open('corpus1.txt', 'r') as f:
    corpus1 = f.read()

with open('corpus2.txt', 'r') as f:
    corpus2 = f.read()

corp1 = corpus1.split('\n\n')
corp2 = corpus2.split('\n\n')
print(len(corp1))
print(len(corp2))

67937
2528


In [3]:
# delete punctuation
# re.sub(pattern, repl, string, count=0, flags=0)
# put words in englishST into a list
with open('englishST.txt', 'r') as eng:
    eng_str = eng.read()
eng_list = eng_str.split()
    
r = """[0-9!#$%&'"()*+,-./:;\\\<=>?@[\]^_`{|}~\n]"""


def preprocessing(s):
    # delete punctuation
    no_punct = re.sub(r, ' ', s)

    # transform to lower_case
    no_punct = no_punct.lower()

    # put string to words list
    no_list = no_punct.split()

    # delete stopping words(englishST.txt) in no_list
    stop_list = []
    for i in no_list:
        if (i not in eng_list):
            stop_list.extend([i])
    stop_list

    # stemming
    norm_list = []
    for i in stop_list:
        norm_list.append(stem(i))
    
    return norm_list

In [4]:
# get word_list of corp1 and corp2
# every list in corp1 represents a document
# every list in corp1 represents a document
corp1_list = []
corp2_list = []

for i in range(len(corp1)):
    line = preprocessing(corp1[i])
    if (len(line) > 0):
        corp1_list.extend([line])

for j in range(len(corp2)):
    line = preprocessing(corp2[j])
    if (len(line) > 0):
        corp2_list.extend([line])

In [5]:
print("number of documents in corp1: "+ str(len(corp1_list)))
print("number of documents in corp2: " + str(len(corp2_list)))

number of documents in corp1: 66420
number of documents in corp2: 2458


In [6]:
# get corpus dictionary and its key_list

# opeartion for corpus1
corpus1_list = preprocessing(corpus1)
corpus1_counter = Counter(corpus1_list)
corpus1_dict = dict(corpus1_counter)
# operation for corpus2
corpus2_list = preprocessing(corpus2)
corpus2_counter = Counter(corpus2_list)
corpus2_dict = dict(corpus2_counter)

# delete when value < 10
list1 = []; list2 = []
for key, value in corpus1_dict.items():
    if (key in corpus2_dict.keys()):
        if (corpus2_dict[key] + value >= 10):
            list1.append(key)
            list2.append(value)
    else:
        if (value >= 10):
            list1.append(key)
            list2.append(value)
            
corpus1_dict = dict(zip(list1,list2))
corpus1_key = list(corpus1_dict.keys())

# delete when value < 10
list1 = []; list2 = []
for key, value in corpus2_dict.items():
    if (key in corpus1_dict.keys()):
        if (corpus1_dict[key] + value >= 10):
            list1.append(key)
            list2.append(value)
    else:
        if (value >= 10):
            list1.append(key)
            list2.append(value)
            
corpus2_dict = dict(zip(list1,list2))
corpus2_key = list(corpus2_dict.keys())

In [7]:
# special word list in corpus1 and corpus2

special_list = list(set(corpus1_key).union(set(corpus2_key)))
print('length of corpus1_key: ' + str(len(corpus1_key)))
print('length of corpus2_key: ' + str(len(corpus2_key)))
print('length of special_list: ' + str(len(special_list)))

length of corpus1_key: 9036
length of corpus2_key: 4658
length of special_list: 9039


In [8]:
# calculate the number of times every word appears in the document in each corpus

num_corp1_dict = {}
for i in range(len(corpus1_key)):
    num_corp1_dict[corpus1_key[i]] = 0
    
for i in range(len(corp1_list)):
    w_list = list(np.unique(corp1_list[i]))
    for w in w_list:
        if (w in corpus1_key):
            num_corp1_dict[w] += 1

num_corp2_dict = {}
for i in range(len(corpus2_key)):
    num_corp2_dict[corpus2_key[i]] = 0
    
for i in range(len(corp2_list)):
    w_list = list(np.unique(corp2_list[i]))
    for w in w_list:
        if (w in corpus2_key):
            num_corp2_dict[w] += 1  

In [9]:
len(num_corp2_dict)

4658

In [10]:
# compute mutual information
mi_list = []
X2_list = []

def takeSecond(elem):
    return elem[1]

# calaculate N N0 N1 N11 N10 N01 N00 for corpus1
N = len(corp1_list) + len(corp2_list)  # corpus1 + corpus2
N1 = len(corp1_list)  # corpus1
N0 = len(corp2_list)  # corpus2


for w in special_list:
    if (w in corpus1_key):  # in corpus1
        N11 = num_corp1_dict[w]  
    else:
        N11 = 0
    N01 = N1 - N11
    
    if (w in corpus2_key):  # in corpus2
        N10 = num_corp2_dict[w]    
    else:
        N10 = 0
    N00 = N0 - N10
    

    if (N11 != 0 and N01 != 0 and N10 != 0 and N00 != 0):
        a = (N11 / N) * math.log2((N * N11)/((N10+N11) * N1))
        b = (N01 / N) * math.log2((N * N01)/((N00+N01) * N1))
        c = (N10 / N) * math.log2((N * N10)/((N10+N11) * N0))
        d = (N00 / N) * math.log2((N * N00)/((N00+N01) * N0))
    
    mi = a + b + c + d
    X2 = ((N11+N10+N01+N00)*(N11*N00-N10*N01)**2) / ((N11+N01)*(N11+N10)*(N10+N00)*(N01+N00))
     
    mi_list.extend([[w, mi]])
    X2_list.extend([[w, X2]])
    
mi_list.sort(key=takeSecond, reverse=True)
X2_list.sort(key=takeSecond, reverse=True)
    

In [11]:
# print(mi_list)
# print(X2_list)
a = corp1_list.copy()
a.extend(corp2_list)
len(a)

68878

# TOPIC-LEVEL COMPARISONS

In [12]:
# Create a corpus from a list of texts
common_texts = corp1_list.copy()
common_texts.extend(corp2_list)

common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus
lda = LdaModel(common_corpus, num_topics=10, id2word=common_dictionary)

In [13]:
lda.show_topics()
# lda.print_topics()
# lda.get_topic_terms(0)

[(0,
  '0.028*"car" + 0.026*"sister" + 0.017*"drive" + 0.012*"citi" + 0.010*"time" + 0.010*"uber" + 0.009*"airlin" + 0.009*"servic" + 0.007*"phone" + 0.007*"day"'),
 (1,
  '0.025*"googl" + 0.022*"org" + 0.018*"https" + 0.018*"http" + 0.016*"en" + 0.016*"wikipedia" + 0.015*"wiki" + 0.015*"facebook" + 0.012*"user" + 0.010*"reddit"'),
 (2,
  '0.025*"compani" + 0.017*"peopl" + 0.016*"job" + 0.013*"work" + 0.013*"make" + 0.010*"good" + 0.009*"don" + 0.008*"bonus" + 0.008*"money" + 0.007*"busi"'),
 (3,
  '0.049*"http" + 0.044*"www" + 0.027*"reddit" + 0.026*"comment" + 0.021*"loan" + 0.017*"bailout" + 0.017*"messag" + 0.016*"bank" + 0.013*"diamond" + 0.012*"news"'),
 (4,
  '0.018*"market" + 0.015*"price" + 0.011*"cost" + 0.011*"busi" + 0.008*"product" + 0.008*"peopl" + 0.008*"industri" + 0.008*"make" + 0.007*"profit" + 0.007*"good"'),
 (5,
  '0.033*"govern" + 0.012*"state" + 0.009*"peopl" + 0.008*"countri" + 0.008*"problem" + 0.007*"regul" + 0.007*"privat" + 0.007*"power" + 0.007*"obama" + 0.

In [1]:
# for doc in corp1_list:
doc = corp1_list[0]
doc_bow = common_dictionary.doc2bow(doc)
lda.get_document_topics(doc_bow)

NameError: name 'corp1_list' is not defined

In [85]:
a = {'a': 1, 'b': 2}
'a' in a.keys()

True